# Incremental Ernollment Trial (Session Matching)
---------

여러개의 초기 enrollment의 case도 고려해서 만드려고 한다.

그리고 session을 고려한 case를 만들고싶다.

## Environment

In [11]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

from utils import key2df

In [5]:
sys.path.append('/host/projects/sv_experiments/sv_system')

NameError: name 'sys' is not defined

## Dataframes & embeddings

In [19]:
keys = pickle.load(open("./xvector_embeds/sv_keys.pkl", "rb"))
keys = np.array(keys)
embeds = np.load("./xvector_embeds/sv_embeds.npy")
embeds = embeds / np.linalg.norm(embeds, axis=1, keepdims=True)

In [20]:
key_df = key2df(keys)
key2index = {k:v for v, k in enumerate(keys)}  # it used for mapping from key to index

In [21]:
spk_uttr_stat = key_df.spk.value_counts()

## Held-out validation set

utterance가 150개 넘는 것들을 trial spks로 빼고  
나머지는 validation spk로 빼었다.  
그리고 validation spk 음성을 이용해서 threshold를 정하기 위한 trial을 만든다.

In [22]:
## trial_enr_spks = spk_uttr_stat[spk_uttr_stat > 150].index.tolist()
val_spks = spk_uttr_stat[spk_uttr_stat <= 150].index.tolist()
val_embed_df = key_df[key_df.spk.isin(val_spks)]
trial_embed_df = key_df[key_df.spk.isin(trial_enr_spks)]

In [23]:
voxc2_trial = pd.read_pickle("trials/voxc2_1211_trials.pkl")
trial_for_thresh = voxc2_trial[(voxc2_trial.enroll_spk.isin(val_spks)) & (voxc2_trial.test_spk.isin(val_spks))]

## Design trials for each enr_spk

session을 하나 뽑아서 개 중에서 enr_uttr을 뽑고 target_trial_uttr를 뽑아보자

In [4]:
# session을 매칭시킬지 그렇지 않을지 선택한다.
same_session = False

In [90]:
cases = []
n_enroll_utters = 1
n_target_session = 1


for enr_spk in trial_enr_spks:
    target_sessions = trial_embed_df[trial_embed_df.spk == enr_spk].session.unique()
    for i, sess in enumerate(target_sessions):
        target_uttrs = trial_embed_df[(trial_embed_df.spk == enr_spk) & (trial_embed_df.session == sess)] 
        enr_uttr_pool = target_uttrs.sample(n=1*n_enroll_utters) 
        if not same_session:           
            target_sessions_set = set(target_sessions)
            target_sessions_set.remove(sess)
            target_session = random.choice(list(target_sessions_set))
            target_uttrs = trial_embed_df[(trial_embed_df.spk == enr_spk) & (trial_embed_df.session == target_session)]
        if len(target_uttrs) < 5:
            continue
        nonTarget_uttrs = trial_embed_df[trial_embed_df.spk != enr_spk]
        # we have target ratios [0.9, 0.5, 0.1, 0.01]
        nonTarget_n_uttrs = (np.array(len(target_uttrs)-n_enroll_utters)*np.array([1/9, 1, 9])).astype(np.int64)
        # for p_ratio is 0.01 
        nonTarget_n_uttrs = np.append(nonTarget_n_uttrs, min(len(nonTarget_uttrs), 99*len(target_uttrs)))
        for i in range(0, len(enr_uttr_pool), n_enroll_utters):
            enr_uttrs = enr_uttr_pool[i:i+n_enroll_utters]
            for n_imp_uttr in nonTarget_n_uttrs: # neg_trial ratio 
                if same_session:
                    target_trial_uttrs = target_uttrs.drop(index=enr_uttrs.index)
                else:
                    target_trial_uttrs = target_uttrs
                nonTarget_trial_uttrs = nonTarget_uttrs.sample(n=n_imp_uttr)
                cases += [[enr_spk, 
                           enr_uttrs.index.tolist(), 
                           target_trial_uttrs.index.tolist(),
                           nonTarget_trial_uttrs.index.tolist()]]

sorted_cases = sorted(cases, key=lambda x: len(x[3])) 

## Split trials according to pos_ratio

In [91]:
neg_legnth = list(map(lambda x: len(x[3]), cases))

In [92]:
pos_ratio_0 = cases[0:len(cases):4] # 0.9
pos_ratio_1 = cases[1:len(cases):4] # 0.5
pos_ratio_2 = cases[2:len(cases):4] # 0.1
pos_ratio_3 = cases[3:len(cases):4] # 0.01

## Save the trials

In [95]:
save_dir = "./trials/enr306_session_diff/"
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
trial_for_thresh.to_pickle(save_dir + "/trial_for_thresh.pkl")
val_embed_df.to_pickle(save_dir + "/valdiation_df.pkl")
pickle.dump(pos_ratio_0, open(save_dir + "/trials_ratio_0.9.pkl", "wb"))
pickle.dump(pos_ratio_1, open(save_dir + "/trials_ratio_0.5.pkl", "wb"))
pickle.dump(pos_ratio_2, open(save_dir + "/trials_ratio_0.1.pkl", "wb"))
pickle.dump(pos_ratio_3, open(save_dir + "/trials_ratio_0.01.pkl", "wb"))

In [94]:
cases[1]

['id10986',
 ['id10986-33NaEPsiFEA-00002'],
 ['id10986-s2077pq_vdY-00001',
  'id10986-s2077pq_vdY-00002',
  'id10986-s2077pq_vdY-00003',
  'id10986-s2077pq_vdY-00004',
  'id10986-s2077pq_vdY-00005',
  'id10986-s2077pq_vdY-00006'],
 ['id10399-JL9kpRAbMno-00010',
  'id11075-X1Kwj4fIAbo-00020',
  'id11027-y0qy4sifQRY-00002',
  'id11211-suSnxN1Wm8c-00031',
  'id11181-XsiRQ8XiWOk-00009']]